# Задание на coursera #

## Анализ текстов ##

Цель:
* находить оптимальные параметры для метода опорных векторов
* работать с текстовыми данными

### Данные: ###
    В этом задании мы применим метод опорных векторов для определения того, к какой из тематик относится новость: атеизм или космос.
    Данные взяты в библиотеке Scikit-learn

In [5]:
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.svm import SVC

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [6]:
newsgroups = datasets.fetch_20newsgroups(
                    subset='all', 
                    categories=['alt.atheism', 'sci.space']
             )

In [7]:
X = newsgroups.data
y = newsgroups.target

**подготовим числовое представление данных**

In [8]:
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X)

**Реализация SVM-классификатора находится в классе sklearn.svm.SVC. Веса каждого признака у обученного классификатора хранятся в поле coef_. Чтобы понять, какому слову соответствует i-й признак, можно воспользоваться методом get_feature_names() у TfidfVectorizer:**

In [12]:
feature_names = np.asarray(vectorizer.get_feature_names())
feature_names

array(['00', '000', '0000', ..., 'zwarte', 'zwork', 'zyklon'],
      dtype='<U80')

**Подбор параметров удобно делать с помощью класса sklearn.grid_search.GridSearchCV. Первым аргументом в GridSearchCV передается классификатор, для которого будут подбираться значения параметров, вторым — словарь (dict), задающий сетку параметров для перебора.**

In [13]:
grid = {'C': np.power(10.0, np.arange(-5, 6))}
cv = KFold(n_splits=5, shuffle=True, random_state=241)
clf = SVC(kernel='linear', random_state=241)
gs = GridSearchCV(clf, grid, scoring='accuracy', cv=cv)
gs.fit(X_train, y)

GridSearchCV(cv=KFold(n_splits=5, random_state=241, shuffle=True),
             estimator=SVC(kernel='linear', random_state=241),
             param_grid={'C': array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02,
       1.e+03, 1.e+04, 1.e+05])},
             scoring='accuracy')

**После того, как перебор окончен, можно проанализировать значения качества для всех значений параметров и выбрать наилучший вариант:**

In [14]:
df = pd.DataFrame.from_dict(gs.cv_results_)
df.head()
df_with_best_score = df[df['mean_test_score'] == (df['mean_test_score'].max())]
best_C = df_with_best_score['param_C'].max()
best_C

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,3.329302,0.195761,0.826446,0.079958,1e-05,{'C': 1e-05},0.544693,0.579832,0.571429,0.501401,0.565826,0.552636,0.028124,8
1,3.354806,0.100692,0.773078,0.040115,0.0001,{'C': 0.0001},0.544693,0.579832,0.571429,0.501401,0.565826,0.552636,0.028124,8
2,3.376312,0.185487,0.739429,0.039999,0.001,{'C': 0.001},0.544693,0.579832,0.571429,0.501401,0.565826,0.552636,0.028124,8
3,3.166344,0.115688,0.720361,0.017600,0.01,{'C': 0.01},0.544693,0.579832,0.571429,0.501401,0.565826,0.552636,0.028124,8
4,2.582100,0.059262,0.610412,0.021789,0.1,{'C': 0.1},0.958101,0.949580,0.957983,0.935574,0.949580,0.950164,0.008218,7


100000.0

**Обучите SVM по всей выборке с оптимальным параметром C, найденным на предыдущем шаге.**

In [15]:
clf.set_params(C = best_C).fit(X_train, y)

SVC(C=100000.0, kernel='linear', random_state=241)

In [ ]:
Найдите 10 слов с наибольшим абсолютным значением веса (веса хранятся в поле coef_ у svm.SVC). Они являются ответом на это задание.